1. 安装pytorch geometric和tensorboardX

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110  torchaudio===0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-geometric 
!pip install tensorboardX
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

2. 导入需要的包，分别是：pytorch、pytorch geometric、tensorboardX

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid

# 增加实验所使用的数据集
from torch_geometric.datasets import Flickr
from torch_geometric.datasets import Reddit2

from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


3. 定义模型 

使用pytorch geometric自定义图神经网络层构建图模型


In [ ]:
class GNNStack(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, task='node'):
        super(GNNStack, self).__init__()
        self.task = task
        
        # 其实这里可以定义一个pre-message-passing 将输入从input_size变换成hidden_size
        self.pre_mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Dropout(0.25),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.convs = nn.ModuleList()
        # self.convs.append(self.build_conv_model(input_dim, hidden_dim))  # 第一层卷积层
        self.lns = nn.ModuleList()
        for i in range(2):
          self.lns.append(nn.LayerNorm(hidden_dim))  # Layer Norm
        self.bns = nn.ModuleList()
        for i in range(3):
          self.bns.append(pyg_nn.BatchNorm(hidden_dim))   # 每层中都增加一个batch norm
        # for l in range(2):  # 第二层和第三层卷积层
        #     self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))
        for l in range(3):  # 定义三层卷积层
             self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))

        # post-message-passing  最后一个卷积层结束后，经过一个多层感知机模型
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25), 
            nn.Linear(hidden_dim, output_dim))
        if not (self.task == 'node' or self.task == 'graph'):
            raise RuntimeError('Unknown task.')

        self.dropout = 0.25
        self.num_layers = 3  # 在真实的应用中，这些参数都应该为用户自定义超参数

    def build_conv_model(self, input_dim, hidden_dim):
        # refer to pytorch geometric nn module for different implementation of GNNs.
        if self.task == 'node':
            return CustomConv(input_dim, hidden_dim)  # 用户自定义图神经网络
        else:  # 如果是graph 任务，当前最好的图神经网络为GIN
            return pyg_nn.GINConv(nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                  nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)))

    def forward(self, data):  # 定义整个图神经网络的行为
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.pre_mlp(x)  # 开始前调用pre-message-passing函数
        if data.num_node_features == 0:
          x = torch.ones(data.num_nodes, 1)  # 如果初始时，没有给节点分配特征，则初始化每个节点的特征向量为常数1
        
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)  # 调用GNN的forward方法，forward会通过propagate传递来的参数分别启动message和update函数
            emb = x
            x = self.bns[i](x) # 增加batch_norm层
            x = F.dropout(x, p=self.dropout, training=self.training)  # 做了dropout
            pyg_utils.dropout_adj(edge_index, p = self.dropout)  # 每一层都随机dropout一些边，需要做实验看这一不分是否有必要
            x = F.leaky_relu(x)  # 使用leaky_relu激活函数，这里的激活函数可以考虑换成F.prelu
            if not i == self.num_layers - 1:  # 除了最后一层之外，其它层都做了layer norm
                x = self.lns[i](x)

        if self.task == 'graph':
            x = pyg_nn.global_mean_pool(x, batch)

        x = self.post_mp(x)  #调用后post-message-passing

        return emb, F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)

In [ ]:
class CustomConv(pyg_nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = nn.Linear(in_channels, out_channels)
        self.lin_self = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        # Add self-loops to the adjacency matrix.
        edge_index, _ = pyg_utils.add_self_loops(edge_index, num_nodes = x.size(0))

        # Transform node feature matrix.
        self_x = self.lin_self(x)
        #x = self.lin(x)
        
        row, col = edge_index
        deg = pyg_utils.degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]


        return self_x + self.propagate(edge_index, x = x, norm = norm)

    def message(self, x_j, norm):
        # Compute messages
        # x_j has shape [E, out_channels]  # x_j is neighborhood embeddings,  x_i is yourself embeddings
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):  # 这里实际上可以通过多层感知机处理aggr_out，最终的输出会经过aggr='add'
        # aggr_out has shape [N, out_channels]
        return aggr_out

4. 训练

In [ ]:
def train(dataset, task, writer):
    if task == 'graph':
        data_size = len(dataset)
        loader = DataLoader(dataset[:int(data_size * 0.8)], batch_size=64, shuffle=True)
        test_loader = DataLoader(dataset[int(data_size * 0.8):], batch_size=64, shuffle=True)
    else:
        test_loader = loader = DataLoader(dataset, batch_size=64, shuffle=True)

    # build model
    model = GNNStack(max(dataset.num_node_features, 1), 32, dataset.num_classes, task=task)
    opt = optim.Adam(model.parameters(), lr=0.01)
    
    # train
    for epoch in range(200):
        total_loss = 0
        model.train()
        for batch in loader:
            #print(batch.train_mask, '----')
            opt.zero_grad()
            embedding, pred = model(batch)
            label = batch.y
            if task == 'node':
                pred = pred[batch.train_mask]
                label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        writer.add_scalar("loss", total_loss, epoch)

        if epoch % 10 == 0:
            test_acc = test(test_loader, model)
            print("Epoch {}. Loss: {:.4f}. Test accuracy: {:.4f}".format(
                epoch, total_loss, test_acc))
            writer.add_scalar("test accuracy", test_acc, epoch)

    return model


5. 测试

In [ ]:
def test(loader, model, is_validation=False):
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            emb, pred = model(data)
            pred = pred.argmax(dim=1)
            label = data.y

        if model.task == 'node':
            mask = data.val_mask if is_validation else data.test_mask
            # node classification: only evaluate on nodes in test set
            pred = pred[mask]
            label = data.y[mask]
            
        correct += pred.eq(label).sum().item()
    
    if model.task == 'graph':
        total = len(loader.dataset) 
    else:
        total = 0
        for data in loader.dataset:
            total += torch.sum(data.test_mask).item()
    return correct / total

6. 训练模型，并使用tensorboardX可视化loss和accuracy

In [ ]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

# dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
# dataset = Flickr(root='/tmp/FLICKR')
dataset = Reddit2(root='/tmp/Reddit2')
dataset = dataset.shuffle()
task = 'node'
model = train(dataset, task, writer)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir './log'

7. 测试模型

In [ ]:
writer = SummaryWriter("./log" + datetime.now().strftime("%Y%m%d-%H%M%S"))

dataset = Planetoid(root='/tmp/cora', name='cora')
task = 'node'

model = train(dataset, task, writer)

In [ ]:
color_list = ["red", "orange", "green", "blue", "purple", "brown"]

loader = DataLoader(dataset, batch_size=64, shuffle=True)
embs = []
colors = []
for batch in loader:
    emb, pred = model(batch)
    embs.append(emb)
    colors += [color_list[y] for y in batch.y]
embs = torch.cat(embs, dim=0)

xs, ys = zip(*TSNE().fit_transform(embs.detach().numpy()))
plt.scatter(xs, ys, c = colors/255)
plt.show()